
# Importing Libraries, Requirements and Datasets

In [ ]:
#Installer tous les packages nécaissaires
pip install -r ../requirements.txt

In [1]:
import openpyxl
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import nltk
import string                              # for string operations
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer  
from nltk.corpus import stopwords
import spacy
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from collections import Counter
import string
from sklearn.model_selection import train_test_split


nltk.download('stopwords')
stopwords_french = stopwords.words('french')
print('Stop words\n')
print(stopwords_french)


Stop words

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'a

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bedhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

# Chemins des fichiers XLSX
file1 = '../resources/common/data/labled_comments.xlsx'
file2 = '../resources/common/data/all_raw_comments_cleaning.xlsx'
# Lecture des fichiers XLSX
dflabled = pd.read_excel(file1)
dfnew_comment= pd.read_excel(file2)


In [3]:

# Assurez-vous d'avoir le modèle français de spaCy installé
# Vous pouvez l'installer en utilisant : python -m spacy download fr_core_news_sm
nlp = spacy.load('fr_core_news_sm')

# Liste des stopwords français
stopwords_french = set(stopwords.words('french'))

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'^RT[\s]+', '', text)
        text = re.sub(r'https?://[^\s\n\r]+', '', text)
        text = re.sub(r'#', '', text)
        text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '', text)
        text = re.sub(r'\b\d{2}:\d{2}(:\d{2})?\b', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'^(\s*\.?\s*)$', '', text, flags=re.MULTILINE)
        text = unidecode(text)
    else:
        if isinstance(text, (int, float)):
            text = str(text)
        elif pd.isnull(text):
            text = ''
    return text.strip()

def clean_and_process_text(df, column_name):
    # Appliquer clean_text à la colonne spécifiée
    df[column_name] = df[column_name].apply(clean_text)
    
    # Remplacer les chaînes vides par NaN et supprimer les lignes avec NaN
    df.replace("", np.nan, inplace=True)
    df.dropna(subset=[column_name], inplace=True)
    
    # Tokenisation
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    df['tokens'] = df[column_name].apply(tokenizer.tokenize)
    
    # Suppression des stopwords et ponctuation
    df_clean = []
    for tokens in df['tokens']:
        clean_tokens = [word for word in tokens if word == 'pas' or word == 'ne' or (word not in stopwords_french and word not in string.punctuation)]
        df_clean.append(clean_tokens)
    
    # Ajouter les tokens nettoyés comme une nouvelle colonne
    df['clean_tokens'] = df_clean
    
    # Lemmatisation et autres remplacements
    df_lemmatized = []
    for tokens in df_clean:
        lemmatized_tokens = []
        for word in tokens:
            doc = nlp(word)
            lemma = doc[0].lemma_ if doc else word
            lemmatized_tokens.append(lemma)
        
        # (Ici vous pouvez ajouter d'autres remplacements comme vous avez montré dans votre code# (suite du code)
        text_str = ' '.join(lemmatized_tokens)
        
        # Appliquer divers remplacements
        text_str = re.sub(r'\bprescr\w*\b', 'prescrire', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\bsat\b|\bst\b|\bsatisfaire\b|\bsatisfaisant\b|\bsatisfaites\b', 'satisfait', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\brap(el|elle)\b', 'rappel', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\bbc\b|\bbcp\b', 'beaucoup', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\baimer\b|\baime\b', 'aimer', text_str, flags=re.IGNORECASE)
        #text_str = re.sub(r'\baucun\b|\baucune\b|\bne\b|\bpas\b|\babsence\b|\babsant\b|\bnon\b', 'negative', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\btre\b', 'tres', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\bretou\b', 'retour', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\blindication\b', 'indication', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\bpresentationell\b', 'presentation', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\blutilis\b', 'utiliser', text_str, flags=re.IGNORECASE)
        text_str = re.sub(r'\blachete\b', 'acheter', text_str, flags=re.IGNORECASE)
        
        # Ajouter à la liste
        lemmatized_tokens = text_str.split()
        df_lemmatized.append(lemmatized_tokens)
    
    # Ajouter les tokens lemmatisés comme une nouvelle colonne
    df['lemmatized_tokens'] = df_lemmatized
   

    # Lire le fichier mots.csv et créer un ensemble de mots à supprimer
    words_to_remove = set()
    with open('../resources/common/data/600_mots_moins_freq.csv', 'r') as file:
        for line in file:
            word = line.strip()  # supprime les espaces en début et en fin de ligne
            words_to_remove.add(word)
    
    # Pour chaque ligne de df['lemmatized_tokens'], supprimez les mots présents dans mots.csv
    def filter_tokens(tokens):
        return [token for token in tokens if token not in words_to_remove]
    
    # Appliquer la fonction de filtrage à chaque ligne de lemmatized_tokens
    df['lemmatized_tokens_filtred'] = df['lemmatized_tokens'].apply(filter_tokens)
    
    return df


In [4]:
df_processed_labled = clean_and_process_text(dflabled, 'comment')
df_processed_labled 

,comment,score,tokens,clean_tokens,lemmatized_tokens,lemmatized_tokens_filtred
0,aussi absorption directement niveau peau dit p...,Réclamation,"[aussi, absorption, directement, niveau, peau,...","[aussi, absorption, directement, niveau, peau,...","[aussi, absorption, directement, niveau, peau,...","[absorption, directement, niveau, dire, pourqu..."
1,doute plus pour efficacite produit,Réclamation,"[doute, plus, pour, efficacite, produit]","[doute, plus, efficacite, produit]","[doute, plus, efficacite, produit]","[doute, plus, efficacite, produit]"
2,jai reclame regler situation,Refus,"[jai, reclame, regler, situation]","[jai, reclame, regler, situation]","[jai, reclame, regler, situation]","[reclame, regler, situation]"
3,jamais recu echantillon auparavant,Refus,"[jamais, recu, echantillon, auparavant]","[jamais, recu, echantillon, auparavant]","[jamais, recu, echantillon, auparavant]","[jamais, recu, echantillon, auparavant]"
4,meme sil observe antecedent cheloide eme jour,Réclamation,"[meme, sil, observe, antecedent, cheloide, eme...","[meme, sil, observe, antecedent, cheloide, eme...","[meme, sil, observer, antecedent, cheloide, em...","[meme, observer]"
...,...,...,...,...,...,...
2569,vient prescrire,Client,"[vient, prescrire]","[vient, prescrire]","[venir, prescrire]","[venir, prescrire]"
2570,vient prescrire moment visite,Client,"[vient, prescrire, moment, visite]","[vient, prescrire, moment, visite]","[venir, prescrire, moment, visite]","[venir, prescrire, moment, visite]"
2571,visite rappel,Présentation,"[visite, rappel]","[visite, rappel]","[visite, rappel]","[visite, rappel]"
2572,voi cest probleme essentiellement gastrique,Réclamation,"[voi, cest, probleme, essentiellement, gastrique]","[voi, cest, probleme, essentiellement, gastrique]","[voi, cest, probleme, essentiellement, gastrique]","[voi, probleme, essentiellement, gastrique]"


In [5]:
df_processed_new = clean_and_process_text(dfnew_comment, 'comment')
df_processed_new

,comment,Unnamed: 1,tokens,clean_tokens,lemmatized_tokens,lemmatized_tokens_filtred
0,Bon retour,NaN,"[bon, retour]","[bon, retour]","[bon, retour]","[bon, retour]"
1,Insistance,NaN,[insistance],[insistance],[insistance],[insistance]
2,Insistance,NaN,[insistance],[insistance],[insistance],[insistance]
3,Rappel,NaN,[rappel],[rappel],[rappel],[rappel]
4,Elle a un bon retour sur produit,NaN,"[elle, a, un, bon, retour, sur, produit]","[a, bon, retour, produit]","[avoir, bon, retour, produit]","[avoir, bon, retour, produit]"
...,...,...,...,...,...,...
69892,Insistance,NaN,[insistance],[insistance],[insistance],[insistance]
69893,Insistance,NaN,[insistance],[insistance],[insistance],[insistance]
69894,Insistance,NaN,[insistance],[insistance],[insistance],[insistance]
69895,Insistance,NaN,[insistance],[insistance],[insistance],[insistance]


Modeling

# Semi-supervised Learning with LSTM

In [6]:
np.random.seed(42)

# Étape 1: Entraînez un modèle sur les données étiquetées

small_texts = df_processed_labled['lemmatized_tokens_filtred'].values
small_labels = dflabled['score'].values

# Encodage des étiquettes
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(small_labels)
small_y = to_categorical(encoded_labels)

# Tokenization et padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(small_texts)
sequences = tokenizer.texts_to_sequences(small_texts)
word_index = tokenizer.word_index
small_X = pad_sequences(sequences)

# Construction du modèle
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=small_X.shape[1]))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(small_y.shape[1], activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraînement du modèle
model.fit(small_X, small_y, epochs=10, batch_size=32)

# Étape 2: Utilisez le modèle pour prédire des étiquettes pour les données non étiquetées

# Supposons que df soit votre DataFrame contenant les données non étiquetées
# et que la colonne 'lemmatized_tokens' contient les textes déjà nettoyés, tokenisés et lemmatisés

# Recombiner les tokens en texte
unlabeled_texts = [' '.join(tokens) for tokens in df_processed_new['lemmatized_tokens_filtred'].values]
unlabeled_sequences = tokenizer.texts_to_sequences(unlabeled_texts)
unlabeled_X = pad_sequences(unlabeled_sequences, maxlen=small_X.shape[1])

# Prédiction
predictions = model.predict(unlabeled_X)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels)

# Étape 3: Combinez les données étiquetées originales avec les données auto-étiquetées
combined_texts = np.concatenate((small_texts, unlabeled_texts))
combined_labels = np.concatenate((small_labels, predicted_labels))

# Encodage des étiquettes combinées
encoded_combined_labels = label_encoder.transform(combined_labels)
combined_y = to_categorical(encoded_combined_labels)

# Tokenization et padding des textes combinés
combined_sequences = tokenizer.texts_to_sequences(combined_texts)
combined_X = pad_sequences(combined_sequences, maxlen=small_X.shape[1])

# Étape 4: Ré-entraînez le modèle sur le nouvel ensemble de données combiné
model.fit(combined_X, combined_y, epochs=10, batch_size=32,shuffle=True)
# Préparer les données de la grande dataset pour la prédiction
# Comme précédemment mentionné, 'lemmatized_tokens' contient les textes déjà nettoyés, tokenisés et lemmatisés
large_unlabeled_texts = [' '.join(tokens) for tokens in df_processed_new['lemmatized_tokens_filtred'].values]
large_unlabeled_sequences = tokenizer.texts_to_sequences(large_unlabeled_texts)
large_unlabeled_X = pad_sequences(large_unlabeled_sequences, maxlen=small_X.shape[1])

# Utiliser le modèle pour prédire les étiquettes de la grande dataset
large_predictions = model.predict(large_unlabeled_X)

# Convertir les prédictions en étiquettes lisibles
large_predicted_labels = np.argmax(large_predictions, axis=1)
large_predicted_labels = label_encoder.inverse_transform(large_predicted_labels)

# Ajouter les étiquettes prédites à la grande dataset
df_processed_new['predicted_score_lstm'] = large_predicted_labels
df_processed_new['predicted_score_lstm']

Epoch 1/10
81/81 [==============================] - 2s 10ms/step - loss: 1.7299 - accuracy: 0.3489
Epoch 2/10
81/81 [==============================] - 1s 9ms/step - loss: 1.1751 - accuracy: 0.6002
Epoch 3/10
81/81 [==============================] - 1s 9ms/step - loss: 0.8247 - accuracy: 0.7222
Epoch 4/10
81/81 [==============================] - 1s 9ms/step - loss: 0.6483 - accuracy: 0.7937
Epoch 5/10
81/81 [==============================] - 1s 9ms/step - loss: 0.5816 - accuracy: 0.8069
Epoch 6/10
81/81 [==============================] - 1s 9ms/step - loss: 0.5212 - accuracy: 0.8306
Epoch 7/10
81/81 [==============================] - 1s 9ms/step - loss: 0.4675 - accuracy: 0.8407
Epoch 8/10
81/81 [==============================] - 1s 9ms/step - loss: 0.4265 - accuracy: 0.8473
Epoch 9/10
81/81 [==============================] - 1s 10ms/step - loss: 0.4074 - accuracy: 0.8613
Epoch 10/10
2184/2184 [==============================] - 5s 2ms/step
Epoch 1/10
2265/2265 [=========================

0          Partenaire
1        Présentation
2        Présentation
3        Présentation
4          Partenaire
             ...     
69892    Présentation
69893    Présentation
69894    Présentation
69895    Présentation
69896    Présentation
Name: predicted_score_lstm, Length: 69885, dtype: object

In [7]:
df_processed_new['predicted_score_lstm'].unique()

array(['Partenaire', 'Présentation', 'Client', 'promesse', 'Réclamation',
       'test en cours', 'Refus', 'Livraison'], dtype=object)

In [8]:
df_processed_new.to_excel('../resources/common/resultat.xlsx', index=False)

In [28]:
# Étape 1: Prétraiter la nouvelle donnée
nouvelle_data = "aucune remarque"   # à remplacer par votre texte

# Créer un DataFrame avec une seule colonne 'comment'
df = pd.DataFrame({'comment': [nouvelle_data]})

# Vous devriez effectuer le même prétraitement que pour vos données initiales (par exemple, lemmatisation, etc.)
clean_and_process_text(df, 'comment')

# Vous devriez maintenant extraire la nouvelle donnée prétraitée du DataFrame
nouvelle_data_preprocessed = df.iloc[0]['comment']

# Étape 2: Tokenization et padding
nouvelle_sequence = tokenizer.texts_to_sequences([nouvelle_data_preprocessed])
nouvelle_X = pad_sequences(nouvelle_sequence, maxlen=small_X.shape[1])

# Étape 3: Utilisez le modèle pour faire une prédiction
prediction = model.predict(nouvelle_X)

# Étape 4: Décoder la prédiction pour obtenir l'étiquette correspondante
predicted_label = np.argmax(prediction, axis=1)
predicted_label = label_encoder.inverse_transform(predicted_label)

print(f"Prédiction pour la nouvelle donnée: {predicted_label}")



1/1 [==============================] - 0s 25ms/step
Prédiction pour la nouvelle donnée: ['Présentation']
